### Summary

Results submitted to the contest page on 29 Jan 2020 18:50:18, achieving a $100 \times (1-SMAPE)$ public score of 70.04

In [43]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [412]:
#sns.set_style('whitegrid')
#sns.set(rc={'figure.figsize':(16, 4)})
plt.rc('figure',figsize=(16,4))
plt.rc('font',size=13)

In [439]:
# main dataset to calculate offsets
train = pd.read_pickle('train.pkl')
test = pd.read_pickle('test.pkl')

In [ ]:
trainx = pd.read_csv('train_1.csv', index_col=0)
test = pd.read_csv('test_forR.csv', index_col=0)

In [396]:
train_orig = train.copy()

In [ ]:
trainx_orig = trainx.copy()
test_orig = test.copy()

In [195]:
# to reset 
trainx = trainx_orig.copy()
test = test_orig.copy()

In [113]:
sat_ids = test['sat_id'].unique()

In [196]:
# number of observations in test for each satellite n
num_test = [(test['sat_id'] == n).sum() for n in sat_ids]

In [262]:
# dataframe of frequencies for each satellite, found by R's 'findfrequency'
freq = pd.read_csv('frequencies.csv').drop('Unnamed: 0', axis=1)

In [197]:
freq.head()

,f_x,f_y,f_z,f_Vx,f_Vy,f_Vz
0,24,24,24,12,12,12
1,24,24,24,24,24,24
2,24,24,24,24,24,1
3,24,24,24,24,24,24
4,24,24,24,24,24,24


In [35]:
freq.columns

Index(['f_x', 'f_y', 'f_z', 'f_Vx', 'f_Vy', 'f_Vz'], dtype='object')

In [22]:
freq.shape

(300, 6)

In [198]:
trainx.shape, test.shape

((315183, 4), (284071, 3))

In [199]:
trainx.head()

,id,epoch,sat_id,x
958,1819,2014-01-01 00:00:00.000,1,54678.538321
959,1820,2014-01-01 00:21:11.845,1,55483.832064
960,1821,2014-01-01 00:42:23.690,1,52523.781668
961,1822,2014-01-01 01:03:35.534,1,47848.375008
962,1823,2014-01-01 01:24:47.379,1,42707.748340


In [200]:
test.head()

,id,sat_id,epoch
0,3927,1,2014-02-01 00:01:45.162
1,3928,1,2014-02-01 00:22:57.007
2,3929,1,2014-02-01 00:44:08.852
3,3930,1,2014-02-01 01:05:20.697
4,3931,1,2014-02-01 01:26:32.542


In [201]:
# reindex to epoch 
trainx.index = trainx['epoch']
test.index = test['epoch']

In [202]:
trainx.head()

,id,epoch,sat_id,x
epoch,,,,
2014-01-01 00:00:00.000,1819,2014-01-01 00:00:00.000,1,54678.538321
2014-01-01 00:21:11.845,1820,2014-01-01 00:21:11.845,1,55483.832064
2014-01-01 00:42:23.690,1821,2014-01-01 00:42:23.690,1,52523.781668
2014-01-01 01:03:35.534,1822,2014-01-01 01:03:35.534,1,47848.375008
2014-01-01 01:24:47.379,1823,2014-01-01 01:24:47.379,1,42707.748340


In [443]:
test.head()

,id,sat_id,epoch,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,3927,1,2014-02-01 00:01:45.162,-13366.891347,-14236.753503,6386.774555,4.333815,-0.692764,0.810774
1,3928,1,2014-02-01 00:22:57.007,-7370.434039,-14498.771520,7130.411325,5.077413,0.360609,0.313402
2,3929,1,2014-02-01 00:44:08.852,-572.068654,-13065.289498,7033.794876,5.519106,2.012830,-0.539412
3,3930,1,2014-02-01 01:05:20.697,6208.945257,-9076.852425,5548.296900,4.849212,4.338955,-1.869600
4,3931,1,2014-02-01 01:26:32.542,10768.200284,-2199.706707,2272.014862,1.940505,6.192887,-3.167724


In [204]:
train.head()

,id,epoch,sat_id,x,y,z,Vx,Vy,Vz,x_sim,y_sim,z_sim,Vx_sim,Vy_sim,Vz_sim
0,0,2014-01-01 00:00:00.000,0,-8855.823863,13117.780146,-20728.353233,-0.908303,-3.808436,-2.022083,-8843.131454,13138.221690,-20741.615306,-0.907527,-3.804930,-2.024133
1,1,2014-01-01 00:46:43.000,0,-10567.672384,1619.746066,-24451.813271,-0.302590,-4.272617,-0.612796,-10555.500066,1649.289367,-24473.089556,-0.303704,-4.269816,-0.616468
2,2,2014-01-01 01:33:26.001,0,-10578.684043,-10180.467460,-24238.280949,0.277435,-4.047522,0.723155,-10571.858472,-10145.939908,-24271.169776,0.274880,-4.046788,0.718768
3,3,2014-01-01 02:20:09.001,0,-9148.251857,-20651.437460,-20720.381279,0.715600,-3.373762,1.722115,-9149.620794,-20618.200201,-20765.019094,0.712437,-3.375202,1.718306
4,4,2014-01-01 03:06:52.002,0,-6719.092336,-28929.061629,-14938.907967,0.992507,-2.519732,2.344703,-6729.358857,-28902.271436,-14992.399986,0.989382,-2.522618,2.342237


In [156]:
def calculate_offset(k):
    
    offsets = []
    for n in sat_ids:
        satdata_n = train[train['sat_id'] == n][k]
        off_n = max(satdata_n) - min(satdata_n)
        offsets.append(off_n)
    
    return offsets

In [157]:
# the offsets for x are stored in off_x
off = calculate_offset(k='x')

In [205]:
off[0]

44288.22523152392

**Create a function that takes in x values from january and forecasts for the next month for all 300 satellites,
using the frequencies specified by the freq dataframe**


- grab data
- fit model to that data
- forecast for that data
- deoffset it
- put it into a list
- concatenate all forecast data into a single list and attach it to Test

In [230]:
# ETS(M,A_d,M) is multiplicative error, damped additive trend, multiplicative seasonality
# this makes forecast for all 300 satellites, use forecast_plot to make forecast for one satellite

def forecast(k):
    
    forecast_data = list()
    
    for i in range(0,len(sat_ids)):
        
        n = sat_ids[i]                # n is the satellite ID number
        frequency_n = freq['f_'+k][i] # time series frequency for Satellite n
        offset_n = off_k[i]             # offset for Satellite n
        num_test_n = num_test[i]      # number of observations that will be forecasted for Satellite n
    
        satdata_n = train_k[train_k['sat_id'] == n][k] # data for kinematic state k, Satellite n
    
        fit_n = ExponentialSmoothing(np.asarray(satdata_n), trend='add', seasonal='mul',
                                     seasonal_periods = frequency_n, damped=True).fit()
        
        forecast_n = fit_n.forecast(num_test_n)
                               
        forecast_n -= offset_n
        
        forecast_data.append(forecast_n)
        
        print(f'Forecast of {k} for Satellite {n} has been done.')
    
    return forecast_data

Cell below is a small example to understand how to loop over the satellites. 

In [187]:
# For each satellite number n from the 300 unique satellite IDs in the Test dataset,
#forecast_x = list()

#for i in range(0,2):
    
#    n = sat_ids[i]
    
#    frequency_n = freq['f_'+'x'][i]
#    offset_n = off[i]
#    num_test_n = num_test[i]
    
    # grab nth satellite data
#    satdata_n = trainx[trainx['sat_id'] == n].loc[:,'x']
    
    # fit model to that data
#    fit_n = ExponentialSmoothing(np.asarray(satdata_n), trend = 'add', seasonal = 'mul',
#                                 seasonal_periods = frequency_n, damped = True).fit()
    
    # use model to produce forecast data for the epochs in Test
#    forecast_n = fit_n.forecast(num_test_n)
    
    # deoffset the forecast data by the corresponding amount in 'off'
#    forecast_n -= offset_n
    
    # put the forecast data into a list
#    forecast_x.append(forecast_n)
    

In [192]:
num_test[0:2]

[1901, 375]

In [210]:
#%%time
#forecast_x = forecast(k='x')

Wall time: 8.71 s


In [1]:
%%time
forecast_x = forecast(k='x')

In [217]:
# forecast for remaining kinematic states
rem = 'y z Vx Vy Vz'.split()

In [319]:
# after correcting for seasonality = 1 error, start forecasting from Vx
rem = 'Vx Vy Vz'.split()

In [223]:
forecast_list = list()

In [2]:
%%time
forecast_list = list()

for i in range(0,3):
    
    kinem = rem[i]
    
    train_k = pd.read_csv(f'train_{i+4}.csv', index_col=2)
    
    off_k = calculate_offset(k=kinem)
    
    forecast_k = forecast(k=kinem)
    
    forecast_list.append(forecast_k)

- Seasonal periods must be larger than 1. Forecasts for x, y, z went OK but some seasonal periods for Vx, Vy, Vz must be greater than 1 for the forecast to be done.
- Python raised quite a few ConvergenceWarnings because the optimizer failed to find a local/global minimum. Is this because of too few data points for some of the satellites?


In [251]:
# find which variables has frequency 1
(freq == 1).sum()

f_x      0
f_y      0
f_z      0
f_Vx    10
f_Vy     8
f_Vz    34
dtype: int64

In [313]:
# change them to 2
for k in freq.columns[3:]:
    
    # DO THIS
    freq.loc[freq[k] == 1, k] = 2
    
    # DONT DO THIS
    #freq[freq[k] == 1][k] = 2
    

In [318]:
freq.head()

,f_x,f_y,f_z,f_Vx,f_Vy,f_Vz
0,24,24,24,12,12,12
1,24,24,24,24,24,24
2,24,24,24,24,24,2
3,24,24,24,24,24,24
4,24,24,24,24,24,24


In [321]:
# this forecast_list was after forecasting for y and z
forecast_y = forecast_list[0]

In [322]:
forecast_z = forecast_list[1]

In [358]:
# this forecast_list was after correcting for seasonality = 1 and forecasting for Vx, Vy, Vz
forecast_Vx = forecast_list[0]
forecast_Vy = forecast_list[1]
forecast_Vz = forecast_list[2]

In [325]:
len(forecast_list)

3

In [367]:
len(forecast_x)

300

In [356]:
# concatenate all lists inside each forecast_k
import itertools

In [ ]:
# hats on the forecasted data
x_hat = list(itertools.chain.from_iterable(forecast_x))
y_hat = list(itertools.chain.from_iterable(forecast_y))
z_hat = list(itertools.chain.from_iterable(forecast_z))

In [359]:
Vx_hat = list(itertools.chain.from_iterable(forecast_Vx))
Vy_hat = list(itertools.chain.from_iterable(forecast_Vy))
Vz_hat = list(itertools.chain.from_iterable(forecast_Vz))

In [451]:
test.index = test['epoch']
train.index = train['epoch']

In [444]:
test['x'] = x_hat
test['y'] = y_hat
test['z'] = z_hat
test['Vx'] = Vx_hat
test['Vy'] = Vy_hat
test['Vz'] = Vz_hat

In [381]:
# make a copy for submitting results
test_sub = test.copy()

In [429]:
test_sub.head()

,id,sat_id,epoch,x,y,z,Vx,Vy,Vz
id,,,,,,,,,
3927,3927,1,2014-02-01 00:01:45.162,-24903.661004,-11160.986017,6569.262070,2.358096,-1.464771,1.013617
3928,3928,1,2014-02-01 00:22:57.007,-21164.913439,-12627.051179,7737.702371,2.697827,-1.349015,0.672219
3929,3929,1,2014-02-01 00:44:08.852,-16596.874300,-13618.434230,8526.219117,3.125202,-1.152677,0.242354
3930,3930,1,2014-02-01 01:05:20.697,-11123.695326,-13893.430507,8637.577417,3.649070,-0.810363,-0.276345
3931,3931,1,2014-02-01 01:26:32.542,-4715.815798,-13030.237721,7477.788980,4.169333,-0.207998,-1.048281


In [389]:
submission = test_sub[test.columns[3:]]

In [391]:
submission.head()

,x,y,z,Vx,Vy,Vz
id,,,,,,
3927,-24903.661004,-11160.986017,6569.262070,2.358096,-1.464771,1.013617
3928,-21164.913439,-12627.051179,7737.702371,2.697827,-1.349015,0.672219
3929,-16596.874300,-13618.434230,8526.219117,3.125202,-1.152677,0.242354
3930,-11123.695326,-13893.430507,8637.577417,3.649070,-0.810363,-0.276345
3931,-4715.815798,-13030.237721,7477.788980,4.169333,-0.207998,-1.048281


In [392]:
submission.to_csv('submission.csv')

This was submitted to the contest page on 29 Jan 2020 18:50:18, achieving a $100 \times (1-SMAPE)$ public score of 70.04